In [1]:
# Import the necessary libraries to format, send, and parse our returned results

%matplotlib inline

import birdy
import json
import xarray as xr
import requests
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
import ipyleaflet
import ipywidgets

print(
    'birdy:', birdy.__version__, '\n'
    'json:', json.__version__, '\n'
    'xarray:', xr.__version__, '\n'
    'requests:', requests.__version__, '\n'
    'matplotlib:', matplotlib.__version__, '\n'
    'geopandas:', gpd.__version__, '\n'
    'ipyleaflet:', ipyleaflet.__version__
)

birdy: 0.6.5 
json: 2.0.9 
xarray: 0.14.0 
requests: 2.22.0 
matplotlib: 3.1.1 
geopandas: 0.6.1 
ipyleaflet: 0.11.4


In [2]:
# Ensure that the ipyleaflet extension is enbaled

!jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


In [3]:
# Create WPS instances

# # locally running instance
# raven_url = "http://localhost:9099/wps"
# raven = birdy.WPSClient(raven_url, progress=True)

# instance via pavics.ouranos.ca
pavics_url = "https://pavics.ouranos.ca"
raven_url = f'{pavics_url}/twitcher/ows/proxy/raven/wps'

raven = birdy.WPSClient(raven_url, progress=True)


In [4]:
# Build an interactive map with ipyleaflet

canada_center_lat_lon = (48.63, -74.71)

leaflet_map = ipyleaflet.Map(
    center=canada_center_lat_lon,
    basemap=ipyleaflet.basemaps.OpenTopoMap,
)


zoom_slider = ipywidgets.IntSlider(description='Zoom level:', min=0, max=10, value=4)
ipywidgets.jslink((zoom_slider, 'value'), (leaflet_map, 'zoom'))
widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position='topright')
leaflet_map.add_control(widget_control1)

initial_marker_location = canada_center_lat_lon
marker = ipyleaflet.Marker(location=initial_marker_location, draggable=True)
leaflet_map.add_layer(marker)

leaflet_map

Map(basemap={'url': 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png', 'max_zoom': 17, 'attribution': 'Map da…

In [5]:
user_lonlat = list(reversed(marker.location))
user_lonlat

[-74.71, 48.63]

In [6]:
# geo = raven.hydrobasins_select(location=lonlat, aggregate_upstream=True)
user_location = raven.hydrosheds_select(location=str(user_lonlat), aggregate_upstream=True)


In [7]:
# Extract the URL of the resulting GeoJSON feature
user_shape = user_location.get(asobj=False).feature
user_shape

'https://pavics.ouranos.ca/wpsoutputs/4b3e7934-f434-11e9-acb7-0242ac120006/input.geojson'

In [8]:
# Add this GeoJSON to the map!
user_data = requests.get(user_shape).json()
user_geojson = ipyleaflet.GeoJSON(data=user_data, style = {'color': 'blue', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.5})
leaflet_map.add_layer(user_geojson)

In [9]:
# Let's add a few more regions to the map!

smallwood = raven.hydrosheds_select(location=str([-64.2, 54.1]), aggregate_upstream=True)
erie = raven.hydrosheds_select(location=str([-81.3, 41.95]), aggregate_upstream=True)

In [10]:
# Run the hydrobasins selections
smallwood_shape, erie_shape = smallwood.get(asobj=False).feature, erie.get(asobj=False).feature

# Collect and read in the GeoJSON features
smallwood_data, erie_data = requests.get(smallwood_shape).json(),requests.get(erie_shape).json()

# Add them both to the map
smallwood_geojson = ipyleaflet.GeoJSON(data=smallwood_data, style = {'color': 'magenta', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.5})
erie_geojson = ipyleaflet.GeoJSON(data=erie_data, style = {'color': 'cyan', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.5})

leaflet_map.add_layer(smallwood_geojson)
leaflet_map.add_layer(erie_geojson)
